In [1]:
%%writefile 1.txt
This is a story about cats
our feline pets
Cats are furry animals

Writing 1.txt


In [2]:
%%writefile 2.txt
This story is about surfing
Catching waves is fun
Surfing is a popular water sport

Writing 2.txt


In [3]:
#bulding vocabulary

In [8]:
vocab={}
i=1
with open('1.txt') as f:
    x=f.read().lower().split()

for word in x:
    if word in vocab:
        continue
    else:
        vocab[word]=i
        i+=1
print(vocab)

{'this': 1, 'is': 2, 'a': 3, 'story': 4, 'about': 5, 'cats': 6, 'our': 7, 'feline': 8, 'pets': 9, 'are': 10, 'furry': 11, 'animals': 12}


In [9]:
with open('2.txt') as f:
    x=f.read().lower().split()

for word in x:
    if word in vocab:
        continue
    else:
        vocab[word]=i
        i+=1
print(vocab)

{'this': 1, 'is': 2, 'a': 3, 'story': 4, 'about': 5, 'cats': 6, 'our': 7, 'feline': 8, 'pets': 9, 'are': 10, 'furry': 11, 'animals': 12, 'surfing': 13, 'catching': 14, 'waves': 15, 'fun': 16, 'popular': 17, 'water': 18, 'sport': 19}


In [11]:
#feature extraction
one=['1.txt']+[0]*len(vocab)
one

['1.txt', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [12]:
#count repeated word
with open('1.txt') as f:
    x=f.read().lower().split()
for word in x:
    one[vocab[word]]+=1
one

['1.txt', 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]

In [15]:
two=['2.txt']+[0]*len(vocab)
with open('2.txt') as f:
    x=f.read().lower().split()
for word in x:
    two[vocab[word]]+=1
print(f'{one}\n{two}')

['1.txt', 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]
['2.txt', 1, 3, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 2, 1, 1, 1, 1, 1, 1]


In [16]:
#now bag of words & tf-idf

In [21]:
import numpy as np
import pandas as pd
df=pd.read_csv('smsspamcollection.tsv',sep='\t')
from sklearn.model_selection import train_test_split
x=df['message']
y=df['label']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.3,random_state=42)

In [25]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect=CountVectorizer()
x_train_count=count_vect.fit_transform(x_train)

In [26]:
x_train_count.shape

(3900, 7263)

In [34]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer=TfidfTransformer()
x_train_tfidf=tfidf_transformer.fit_transform(x_train_count)

In [35]:
x_train_tfidf.shape

(3900, 7263)

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
vecto=TfidfVectorizer()
x_train_tfidf=vecto.fit_transform(x_train)

In [37]:
from sklearn.svm import LinearSVC
clf=LinearSVC()
clf.fit(x_train_tfidf,y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [39]:
#now above 3 cell in one 
from sklearn.pipeline import Pipeline
text_clf=Pipeline([('tfidf', TfidfVectorizer()),('clf', LinearSVC())])
text_clf.fit(x_train,y_train)

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

In [43]:
pred=text_clf.predict(x_test)
from sklearn.metrics import confusion_matrix,classification_report
print(confusion_matrix(y_test,pred))
print('\n')
print(classification_report(y_test,pred))

[[1445    3]
 [  10  214]]


             precision    recall  f1-score   support

        ham       0.99      1.00      1.00      1448
       spam       0.99      0.96      0.97       224

avg / total       0.99      0.99      0.99      1672



In [46]:
from sklearn import metrics
metrics.accuracy_score(y_test,pred)

0.9922248803827751

In [51]:
text_clf.predict(['Hi how are you?'])

array(['ham'], dtype=object)